## Importamos librerias



In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import re

## Elección Data Set

### Elegimos el Data Set de Kaggle. https://www.kaggle.com/stefanoleone992/fifa-20-complete-player-dataset

In [26]:
futbol = pd.read_csv("players_20.csv", index_col=0, encoding = "ISO-8859-1")

In [6]:
futbol.head(5)

,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
sofifa_id,,,,,,,,,,,,,,,,,,,,,
158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel AndrÃ©s Messi Cuccittini,32,1987-06-24,170,72,Argentina,FC Barcelona,94,...,68+2,66+2,66+2,66+2,68+2,63+2,52+2,52+2,52+2,63+2
20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,1985-02-05,187,83,Portugal,Juventus,93,...,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3
190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,27,1992-02-05,175,68,Brazil,Paris Saint-Germain,92,...,66+3,61+3,61+3,61+3,66+3,61+3,46+3,46+3,46+3,61+3
200389,https://sofifa.com/player/200389/jan-oblak/20/...,J. Oblak,Jan Oblak,26,1993-01-07,188,87,Slovenia,AtlÃ©tico Madrid,91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183277,https://sofifa.com/player/183277/eden-hazard/2...,E. Hazard,Eden Hazard,28,1991-01-07,175,74,Belgium,Real Madrid,91,...,66+3,63+3,63+3,63+3,66+3,61+3,49+3,49+3,49+3,61+3


### Hacemos un analisis preliminar sobre el tamaño de nuestro data Set. 18278 filas, 103 columnas

In [25]:
futbol.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18278 entries, 158023 to 233449
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   short_name                18278 non-null  object 
 1   age                       18278 non-null  int64  
 2   height_cm                 18278 non-null  int64  
 3   weight_kg                 18278 non-null  int64  
 4   nationality               18278 non-null  object 
 5   club                      18278 non-null  object 
 6   overall                   18278 non-null  int64  
 7   potential                 18278 non-null  int64  
 8   value_eur                 18278 non-null  int64  
 9   player_positions          18278 non-null  object 
 10  preferred_foot            18278 non-null  object 
 11  international_reputation  18278 non-null  int64  
 12  release_clause_eur        16980 non-null  float64
 13  team_jersey_number        18038 non-null  float64
 14  

### Miramos el tamaño nuestro Data Set (filas y columnas sobre las que vamos a trabajar)

In [8]:
futbol.shape

(18278, 103)

### Analizamos las columnas

In [9]:
futbol.columns

Index(['player_url', 'short_name', 'long_name', 'age', 'dob', 'height_cm',
       'weight_kg', 'nationality', 'club', 'overall',
       ...
       'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb'],
      dtype='object', length=103)

### Escogemos las columnas sobre las que vamos a hacer el analisis

In [24]:
futbol = futbol[['short_name','age','height_cm','weight_kg','nationality','club','overall','potential','value_eur','player_positions','preferred_foot','international_reputation','release_clause_eur','team_jersey_number','contract_valid_until']]
futbol.head(5)

,short_name,age,height_cm,weight_kg,nationality,club,overall,potential,value_eur,player_positions,preferred_foot,international_reputation,release_clause_eur,team_jersey_number,contract_valid_until
sofifa_id,,,,,,,,,,,,,,,
158023,L. Messi,32,170,72,Argentina,FC Barcelona,94,94,95500000,"RW, CF, ST",Left,5,195800000.0,10.0,2021.0
20801,Cristiano Ronaldo,34,187,83,Portugal,Juventus,93,93,58500000,"ST, LW",Right,5,96500000.0,7.0,2022.0
190871,Neymar Jr,27,175,68,Brazil,Paris Saint-Germain,92,92,105500000,"LW, CAM",Right,5,195200000.0,10.0,2022.0
200389,J. Oblak,26,188,87,Slovenia,AtlÃ©tico Madrid,91,93,77500000,GK,Right,3,164700000.0,13.0,2023.0
183277,E. Hazard,28,175,74,Belgium,Real Madrid,91,91,90000000,"LW, CF",Right,4,184500000.0,7.0,2024.0


### Ahora tenemos los índices de las columnas sobre las que poder hacer las queries

In [23]:
futbol.columns

Index(['short_name', 'age', 'height_cm', 'weight_kg', 'nationality', 'club',
       'overall', 'potential', 'value_eur', 'player_positions',
       'preferred_foot', 'international_reputation', 'release_clause_eur',
       'team_jersey_number', 'contract_valid_until'],
      dtype='object')

### comprobamos las columnas sobre las que vamos a trabajar

In [12]:
futbol.shape

(18278, 15)

### Analizamos los valores nulos

In [20]:
nulls = futbol.isnull().sum()
nulls

short_name                     0
age                            0
height_cm                      0
weight_kg                      0
nationality                    0
club                           0
overall                        0
potential                      0
value_eur                      0
player_positions               0
preferred_foot                 0
international_reputation       0
release_clause_eur          1298
team_jersey_number           240
contract_valid_until         240
dtype: int64

### Porcentaje de valores que faltan en nuestro scope de trabajo

In [19]:
total_cells = np.product(futbol.shape)
nulls_values = nulls.sum()
percent_missing = (nulls_values / total_cells) * 100
percent_missing

0.6485027537659117

### Porcentaje de valores nulos por columna

In [15]:
column_missing = futbol.isnull().sum() * 100 / futbol.shape[0]
column_missing

short_name                  0.000000
age                         0.000000
height_cm                   0.000000
weight_kg                   0.000000
nationality                 0.000000
club                        0.000000
overall                     0.000000
potential                   0.000000
value_eur                   0.000000
player_positions            0.000000
preferred_foot              0.000000
international_reputation    0.000000
release_clause_eur          7.101433
team_jersey_number          1.313054
contract_valid_until        1.313054
dtype: float64

### Comprobamos la suma de duplicados

In [16]:
futbol.duplicated().sum()

0

### Comprobamos que no haya ningun duplicado

In [17]:
futbol.duplicated()

sofifa_id
158023    False
20801     False
190871    False
200389    False
183277    False
          ...  
245006    False
250995    False
252332    False
251110    False
233449    False
Length: 18278, dtype: bool

## Creamos nuestro nuevo Data Frame limpio(exportamos) sobre el que trabajar


In [18]:

futbol.to_csv('cleandataset/futbol_clean.csv')